<a href="https://colab.research.google.com/github/phumipatc/CU_Submissions/blob/master/AI/Sound_to_Dementia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dataset: DementiaBank**
https://dementia.talkbank.org/


English Pitt Corpus: Cookie theft task
* https://dementia.talkbank.org/access/English/Pitt.html
* Dementia vs control



Preparing environment

In [2]:
!pip install openl3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openl3: filename=openl3-0.4.2-py2.py3-none-any.whl size=249327037 sha256=a0e74b71f5859ad23ede19bace2f46ee6d10c2dddb3b9afe269256851133472e
  Stored in directory: /root/.cache/pip/wheels/d0/4d/0a/e57b1dc8ead91b3c5709d9de4f02d1cdd3a91f609a8f1c1062
  Created wheel for kapre: filename=kapre-0.3.7-py3-none-any.whl size=29608 sha256=77c3ccf83a5059afe9d65db81827730886edb8c4cea7aedc5589b881d9b3c4fd
  Stored in directory: /root/.cache/pip/wheels/3e/54/f9/37e9f36590a5431dc207f7dc0eb2e607ff8958d5728e45ef58
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320711 sh

In [2]:
def tryImportColab():
  try:
    import google.colab
    return True
  except ImportError:
    return False

runningInColab = tryImportColab()
runningInColab

True

In [1]:
if(runningInColab):
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# **Audio Embedding**
# OpenL3
* https://openl3.readthedocs.io/en/latest/tutorial.html
* http://www.justinsalamon.com/uploads/4/3/9/4/4394963/cramer_looklistenlearnmore_icassp_2019.pdf

# AudioSet
* https://github.com/tensorflow/models/tree/master/research/audioset
* Use vggish
* Or, https://tfhub.dev/google/vggish/1

# Other embedding models
* https://tfhub.dev/s?module-type=audio-embedding


In [ ]:
!pwd

/content


In [ ]:
import pandas as pd

address_sample_csv_path = 'sound-dementia-data/ADReSS-M-sample-gr'
if runningInColab:
  address_sample_csv_path = 'drive/MyDrive/' + address_sample_csv_path

# Get dataFrame
address_sample_original_df = pd.read_csv(address_sample_csv_path + "sample-gr-groundtruth.csv")
address_sample_clean_df = address_sample_original_df.dropna().drop_duplicates()

# Cleaning Process
## In gender col, change "Female" to 0 and "Male" to 1
address_sample_clean_df['gender'] = address_sample_clean_df['gender'].apply(lambda x: 0 if x == "Female" else 1)
## In dx column, change "Control" to 0 and "ProbableAD" to 1
address_sample_clean_df['dx'] = address_sample_clean_df['dx'].apply(lambda x: 0 if x == "Control" else 1)

address_sample_clean_df.corr()

<ipython-input-26-f91a83d1feac>:13: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  address_sample_clean_df.corr()


,age,gender,educ,dx,mmse
age,1.000000,-2.211016e-01,0.111367,6.747378e-01,-0.507624
gender,-0.221102,1.000000e+00,0.587643,6.409876e-17,0.137361
educ,0.111367,5.876425e-01,1.000000,-4.240945e-02,0.383415
dx,0.674738,6.409876e-17,-0.042409,1.000000e+00,-0.793052
mmse,-0.507624,1.373606e-01,0.383415,-7.930516e-01,1.000000


In [4]:
import openl3
import os
import soundfile as sf

address_sample_path = 'sound-dementia-data/ADReSS-M-sample-gr/sample-gr'
if runningInColab:
  address_sample_path = 'drive/MyDrive/' + address_sample_path
address_sample_list = []
for fName in os.listdir(address_sample_path):
  address_sample_list.append(address_sample_path + fName)
address_sample_list

['drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs097.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs046.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs047.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs009.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs054.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs002.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs087.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs081.wav']

In [ ]:
result = []
for sample in address_sample_list:
  audio, sr = sf.read(sample)
  embedding, timestamps = openl3.get_audio_embedding(audio, sr)
  result.append([embedding, timestamps])

16/16 [==============================] - 216s 13s/step


In [ ]:
result

# **Classification**
# Classics
* https://scikit-learn.org/stable/supervised_learning.html
* Logistic regression, Support Vector Classification, Decision Tree, Random Forest, Neural Net, AdaBoost, Naïve Bayes
* https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

# Classification heads
* https://www.isca-speech.org/archive/pdfs/interspeech_2021/gauder21_interspeech.pdf
* Neural networks - Conv1D (k=1), Conv1D (k=3), Global. Average
* https://www.isca-speech.org/archive/pdfs/interspeech_2021/wang21ca_interspeech.pdf-Neural networks - Conv - Conv1D - Softmax
* Others
* https://www.tensorflow.org/tutorials/images/transfer_learning#add_a_classification_head

**Classic - Logistic Regressing**

In [ ]:
import sklearn.linear_model